## Procesamiento Datos Pandas

Complete el código faltante según corresponda a manera de lograr automatizar el flujo. Finalmente adjunte una imagen del envio de correo correspondiente.



El archivo candidates.csv contiene 50k filas de datos sobre candidatos. Los campos que estamos usando son:
- First Name
- Last Name
- Email
- Country
- Application Date
- Yoe (years of experience)
- Seniority
- Technology
- Code Challenge Score
- Technical Interview


## 1. Almaceno sobre DB

In [ ]:
# lea el articulo:
# https://medium.com/analytics-vidhya/inserting-an-image-in-an-excel-file-using-pandas-and-sql-e88826680b05


In [ ]:
!pip install xlsxwriter

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

import pandas as pd
import xlsxwriter
import sqlite3
import os


DB = 'candidates.db'
TABLE_NAME = 'candidate'

In [ ]:
def almacenar_pandas_to_sql(df: pd.DataFrame, database_name:str, table_name:str)->None:
  """Procesamiento datos candidatos para almacenarlos sobre db """

  # rename columns
  column_rename = {c: c.replace(' ', '') for c in df.columns}
  df.rename(column_rename,axis='columns', inplace=True)

  # almaceno sobre db
  raise 'No se implemento bd' # debe implementar el guardado a la bd

  #
  sql_table_schema = f'{database_name}.{table_name}'
  cantidad_registros = df.shape[0]
  print(f'Se almacenaron {cantidad_registros} sobre tabla sql {sql_table_schema}')

In [ ]:
# read csv
path = './src/candidates.csv'
df = pd.read_csv(path,sep=';')
df.shape

In [ ]:
df.head(2)

In [ ]:
almacenar_pandas_to_sql(df, DB , TABLE_NAME)

## 2. Procesamiento Datos

Generar un reporte por separado para los paises ( 'United States of America', 'Brazil','Colombia','Ecuador') donde se deberá seleccionar a los candidatos cuyo CodeChallengeScore y TechnicalInterviewScore sea mayor igual a 7. Se deberá crear gráficos integrados sobre cada reporte y además almacenar los datos en un excel con el nombre de cada pais.

In [ ]:
# 1. Lectura de datos
import sqlite3
with sqlite3.connect(DB) as conn:
  df = pd.read_sql_query(f'select * from {TABLE_NAME}', conn)
  pass

In [ ]:
df.head(2)

In [ ]:
# 2. Filtrado de datos
# Se debe filtrar por los paises 'United States of America', 'Brazil','Colombia','Ecuador'
# COmo condicion debe ser CodeChallengeScore >=7 y TechnicalInterviewScore>=7


filterDf = df[(df['Country'].isin(['United States of America', 'Brazil', 'Colombia', 'Ecuador'])) &
              (df['Code Challenge Score'] >= 7) & (df['Technical Interview Score'] >= 7)]

# cree filterDf

filterDf.shape

In [ ]:
filterDf.head(2)

In [ ]:
# - Hires by technology (pie chart)
tec = filterDf.groupby('Technology')['Technology'].count()
tec.plot.pie(figsize=(11, 7))


In [ ]:
# - Hires by seniority (bar chart)
senority_group = filterDf.groupby('Seniority')['Seniority'].count()
senority_group.plot.bar()

### Realizo reporte automatizado

In [ ]:
def genero_grafico_circular(df:pd.DataFrame, country:str)->None:
  """Funcion que se encarga de crear el gráfico en pie"""
  tec = df.groupby('Technology')['Technology'].count()
  tec.plot.pie(figsize=(11, 7))

  plt.savefig(f"./reportes/images/{country}/pie_chart.png",dpi=300, bbox_inches='tight')
  plt.close()
  print('Se genero gráfico circular ...')

In [ ]:
def genero_grafico_barras(df:pd.DataFrame, country:str)->None:
  """Funcion que se encarga de crear el gráfico de barras"""

  senority_group = df.groupby('Seniority')['Seniority'].count()
  senority_group.plot.bar()

  plt.savefig(f"./reportes/images/{country}/bar_chart.png",dpi=300, bbox_inches='tight')
  plt.close()
  print('Se genero gráfico barras ...')

In [ ]:
# genero carpetas necesarias
if not os.path.isdir('./reportes'):
  os.mkdir('reportes')
  os.mkdir('./reportes/images')

#
for country in listado_paises:
  # creo subcarpetas
  if not os.path.isdir(f'./reportes/images/{country}'):
    os.mkdir(f'./reportes/images/{country}')

  # filtro df
  countryDf = filterDf[filterDf['Country']==country]

  # genero gráficos
  genero_grafico_circular(countryDf, country)
  genero_grafico_barras(countryDf, country)

  #Genero reporte Excel por pais
  with pd.ExcelWriter(f"./reportes/{country}.xlsx", engine= "xlsxwriter") as excelBook:

    sheet_name = f"Report-{country}"
    countryDf.to_excel(excelBook, index=False, sheet_name= sheet_name)

    # posiciono sobre hoja de excel
    excel_sheet = excelBook.sheets[sheet_name]

    # almaceno imagen
    image_pie_path = f"./reportes/images/{country}/pie_chart.png"
    image_bar_path = f"./reportes/images/{country}/bar_chart.png"

    excel_sheet.insert_image(1, countryDf.shape[1]+2, image_pie_path) # 1 y df.shape +2 -> establecen posicion de imagen en libro
    excel_sheet.insert_image(countryDf.shape[0]+2, countryDf.shape[1]+2, image_bar_path)

    # Guardo cambios sobre excel
    #excelBook.close()

    print(f'Se generó reporte para el país {country}')
    pass
  print('Se finalizó  la generacion de reportes')

## 3. Envio Correo

In [ ]:
def func_sender_email(**args):
  return NotImplementedError

In [ ]:
# me posiciono sobre carpeta
os.chdir('reportes')

func_sender_email(sender_email='ejemplo@gmail.com',
                sender_password='tupass',
                receiver_email='destinatario@gmail.com',
                subject='Reporte Brazil',
                message='Reporte excel Brazil',
                archivo_adjunto='Brazil.xlsx')

In [29]:
# Configuración del servidor y credenciales
import os 
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

smtp_server = 'smtp.gmail.com'  
smtp_port = 587  
sender_email = 'valecordova.01@gmail.com'
sender_password = 'kjps pqhl xlmz sewr'

# Detalles del correo electrónico
receiver_email ='valecordova@gmail.com'
subject = 'Reporte Brazil'
body = 'Reporte excel Brazil'

# Crear el objeto MIMEMultipart
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = receiver_email
msg['Subject'] = subject
msg.attach(MIMEText(body, 'plain'))

# Adjuntar archivo

file_path = './src/Brazil.xlsx'  # Cambia la ruta al archivo que quieras adjuntar
with open(file_path, 'rb') as file:
    attachment = MIMEApplication(file.read(), _subtype="xlsx")
    attachment.add_header('Content-Disposition', 'attachment', filename=file_path)
    msg.attach(attachment)
    
# Iniciar la conexión con el servidor SMTP
with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()  # Iniciar el modo seguro
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, receiver_email, msg.as_string())

print('Correo enviado exitosamente')

FileNotFoundError: [Errno 2] No such file or directory: './src/Brazil.xlsx'